In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import time
from datetime import datetime

In [255]:
df = pd.read_csv("../boston_kaggle/cab_rides.csv")
df['time_stamp'] = df['time_stamp']/1000
df = df.sort_values(by = "time_stamp")
#make assumption that drivers are driving at 40 mph - get duration in seconds
df['duration'] = df['distance'] / 40 * 60 * 60
df = df.reset_index(drop=True)
df = df.fillna(0)

In [230]:
old_df = df.copy()

In [231]:
df.columns

Index(['distance', 'cab_type', 'time_stamp', 'destination', 'source', 'price',
       'surge_multiplier', 'id', 'product_id', 'name', 'duration'],
      dtype='object')

In [232]:
print(df)

        distance cab_type    time_stamp              destination  \
0           3.03     Lyft  1.543204e+09         Theatre District   
1           1.57     Uber  1.543204e+09                North End   
2           1.30     Uber  1.543204e+09         Theatre District   
3           2.71     Uber  1.543204e+09                   Fenway   
4           2.43     Lyft  1.543204e+09              Beacon Hill   
5           2.71     Uber  1.543204e+09                   Fenway   
6           2.71     Uber  1.543204e+09                   Fenway   
7           2.19     Uber  1.543204e+09                North End   
8           3.05     Uber  1.543204e+09                   Fenway   
9           2.19     Uber  1.543204e+09                North End   
10          2.19     Uber  1.543204e+09                North End   
11          2.22     Lyft  1.543204e+09  Northeastern University   
12          4.46     Lyft  1.543204e+09       Financial District   
13          4.46     Lyft  1.543204e+09       Fi

In [233]:
df.iloc[100]

distance                                            1.45
cab_type                                            Lyft
time_stamp                                   1.54321e+09
destination                                       Fenway
source                                          Back Bay
price                                               10.5
surge_multiplier                                       1
id                  e57bd38c-cb92-414a-be2f-322beb26e8ad
product_id                                     lyft_plus
name                                             Lyft XL
duration                                           130.5
Name: 100, dtype: object

In [256]:
time_start = df['time_stamp'].iloc[100]
time_end = time_start + 60 * 60
df = df[(df['time_stamp'] > time_start) & (df['time_stamp'] < time_end)]

In [282]:
running_price = 0
duration = 0
wait_noise = np.random.normal(0, 1) * 60 * 3 + 5 * 60
print(wait_noise)

time_start = df['time_stamp'].iloc[100]

source = "Haymarket Square"
drives = df.copy()

while(len(drives) > 0):
    running_price += drives['price'].iloc[0]
    duration = df['duration'].iloc[0]
    
    #start time has noise
    time_start = time_start + wait_noise + duration
    
    print("[Time_Start : %s] \t $%s \t [%s] to [%s] " %(str(datetime.fromtimestamp(time_start)).split(".")[0], 
                                                drives['price'].iloc[0],
                                                drives['source'].iloc[0], 
                                                drives['destination'].iloc[0] 
                                                ))
    
    # the next source is the current drive's destination
    source = drives['destination'].iloc[0]
    drives = df[(df['time_stamp'] > time_start) & (df['source'] == source)].reset_index(drop= True)
    
print("Total Price $", running_price)
print("Total Earned $ %.2f" % (running_price * 0.4))

    


264.21173473808756
[Time_Start : 2018-11-25 23:48:35] 	 $16.5 	 [Theatre District] to [North End] 
[Time_Start : 2018-11-25 23:55:33] 	 $16.0 	 [North End] to [North Station] 
[Time_Start : 2018-11-26 00:02:32] 	 $15.0 	 [North Station] to [South Station] 
[Time_Start : 2018-11-26 00:09:30] 	 $31.5 	 [South Station] to [Beacon Hill] 
[Time_Start : 2018-11-26 00:16:28] 	 $9.5 	 [Beacon Hill] to [Boston University] 
[Time_Start : 2018-11-26 00:23:26] 	 $7.0 	 [Boston University] to [Back Bay] 
[Time_Start : 2018-11-26 00:30:24] 	 $23.5 	 [Back Bay] to [North End] 
[Time_Start : 2018-11-26 00:37:22] 	 $30.0 	 [North End] to [Theatre District] 
Total Price $ 149.0
Total Earned $ 59.60


In [281]:
drives

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,duration


In [262]:
str(datetime.fromtimestamp(time_start)).split(".")[0]

'2018-11-26 00:40:18'

In [263]:
datetime.fromtimestamp(int(ts/1000) + 60)

datetime.datetime(2018, 12, 17, 13, 26, 12)

In [264]:
ts/1000

1545071112.138

In [62]:
ts = df['time_stamp'][5]